In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 14.1MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
     |████████████████████████████████| 901kB 33.1MB/s 


In [ ]:
import pandas as pd
import random

In [ ]:
train_data = pd.read_csv("/content/training_small.txt",sep='\t',header=None)
test_data = pd.read_csv("/content/test_small.txt",sep='\t',header=None)

In [ ]:
train_data

,0,1,2
0,3083357,"15 Make space Devote a room, or part of a room...",1
1,3199322,Into the pan.,1
2,3278060,The goddess greeted them at the gates with bre...,1
3,3255013,"PHOTO (COLOR): Bachelor Life #4, (Woman in Bed...",1
4,2086284,"But, it was the most terrifying experience tha...",0
...,...,...,...
8905,3308806,Two of the respondents addressed this issue.,1
8906,2182477,"When I see the picture, It is very wonderful.",0
8907,3232848,"Yet, every year, you fail to give credit to th...",1
8908,2175952,I thing the most worrying things is leaked per...,0


In [ ]:
train = train_data[1]
train_label= train_data[2]
test=test_data[1]
test_label =test_data[2]

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
class NativeDataSet(Dataset):
    def __init__(self, source, target, tokenizer, input_length, output_length):         
        self.source=source
        self.target = target
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
  
    def __len__(self):
        return len(self.target)
    
    # def clean_text(self, text):
    #     text = text.replace('~', '')
    #     return text
      
    def __getitem__(self, index):
        source_i = self.tokenizer.encode_plus(self.source[index], max_length=self.output_length, 
                                                padding='max_length', truncation=True, return_tensors="pt")
        # target_i = torch.tensor([self.target[index][0]])

        target_i = torch.tensor([self.target[index]])
        source_ids = source_i["input_ids"].squeeze()
        src_mask    = source_i["attention_mask"].squeeze()

        return {"input_ids": source_ids, "attention_mask": src_mask, "labels": target_i}

In [ ]:
from transformers import ElectraTokenizer
#, BartTokenizer, GPT2Tokenizer, BertTokenizer, 

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
# tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2")
# tokenizer = BartTokenizer.from_pretrained("hyunwoongko/kobart")

train_dataset = NativeDataSet(train,train_label, tokenizer, 16, 16)
test_dataset = NativeDataSet(test,test_label, tokenizer, 16, 16)

In [ ]:
# for i in range(len(train_dataset)):
#     # print(tokenizer.decode(train_dataset[i]['input_ids']))
#     print(train_dataset[i])

In [ ]:
from transformers import ElectraForSequenceClassification
# AlbertForSequenceClassification, BertForSequenceClassification, DistilBertForSequenceClassification, BartForSequenceClassification, GPT2ForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
def choose_model(model_name):
    if model_name=='electra': return ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')
    elif model_name=='bert': return BertForSequenceClassification.from_pretrained('bert-base-uncased')
    elif model_name=='roberta': return RobertaForSequenceClassification.from_pretrained('roberta-base')
    elif model_name=='albert': return AlbertForSequenceClassification.from_pretrained('albert-base-v2')
    elif model_name=='longformer': LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096')
    elif model_name=='bart': return BartForSequenceClassification.from_pretrained('facebook/bart-large')
    elif model_name=='xlnet': return XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')
    else: 
        print("model not in list")
        return 0

In [ ]:
model = choose_model('electra').to('cuda')

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    # learning_rate= 1e-4,
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
200,0.424200,0.342190,0.881808
400,0.262800,0.232606,0.923523
600,0.195600,0.274553,0.908459
800,0.134500,0.227161,0.924681
1000,0.086500,0.267069,0.923523
1200,0.075200,0.358933,0.914253
1400,0.061000,0.364944,0.917729
1600,0.036400,0.351737,0.921205
1800,0.035300,0.386861,0.925840
2000,0.020000,0.428444,0.929316


TrainOutput(global_step=2790, training_loss=0.09913177528688985, metrics={'train_runtime': 1389.4921, 'train_samples_per_second': 2.008, 'total_flos': 7885050380928.0, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1482752, 'train_mem_gpu_alloc_delta': 108400640, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 114732544})

In [ ]:
trainer.evaluate()
# evaluate the current model after training

{'epoch': 10.0,
 'eval_accuracy': 0.9246813441483198,
 'eval_loss': 0.22716061770915985,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 9062400,
 'eval_runtime': 3.3949,
 'eval_samples_per_second': 254.201}

In [ ]:
# # saving the fine tuned model & tokenizer
# model_path = "/content/gdrive/MyDrive/nativeness-classification"
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=216, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax()

In [ ]:
# Example #1
text = "I'm fine. Thank you and you?"
print(get_prediction(text))

tensor(0, device='cuda:0')
